<a href="https://colab.research.google.com/github/Byeon-MJ/Computer_Vision_Project/blob/main/Video_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# #cmake install
# !pip install cmake

In [ ]:
# # dlib install
# !pip install dlib

In [ ]:
# face_recognition install
!pip install face_recognition

# Face Encoding

In [ ]:
# 학습시킬 이미지 dataset 경로 지정
dataset_paths = ['/content/gdrive/MyDrive/CV/Face Recognition/dataset/Chun/']

names = ['Chun']       # name 지정
number_images = 10      # 학습시킬 이미지 수
image_type = '.jpg'     # 이미지 확장자 지정
encoding_file = 'encodings.pickle'  # 128개의 vector 값 저장할 파일명
model_method = 'cnn'    # cnn or hog. CNN 은 정확도가 높지만 느리고, hog는 빠르지만 정확도가 떨어짐

In [ ]:
knownEncodings = []     # 특성값 저장할 배열 지정
knownNames = []         # name을 저장할 배열 지정

for (i, dataset_path) in enumerate(dataset_paths):
    # person name을 names에서 추출
    name = names[i]

    for idx in range(number_images):
        file_name = dataset_path + str(idx + 1) + image_type

        image = cv2.imread(file_name)

        # 이미지를 face_recognition 에서 처리하기 위해 RGB 형식으로 변경
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 얼굴을 detection하고 boxing
        boxes = face_recognition.face_locations(rgb, model = model_method)

        # 얼굴 영역(boxing Area)만 encoding 계산
        encodings = face_recognition.face_encodings(rgb, boxes)

        # loop over the encodings
        for encoding in encodings:
            print(file_name, name, encoding)
            knownEncodings.append(encoding)
            knownNames.append(name)

In [ ]:
# encoding과 name을 Dictionary 타입으로 저장
data = {"encodings": knownEncodings, "names": knownNames}
f = open(encoding_file, 'wb')
f.write(pickle.dumps(data))
f.close()

# Face Recognition

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import face_recognition
import pickle
import time

In [ ]:
# 원본 동영상
file_name = '/content/gdrive/MyDrive/CV/Face Recognition/chun_video2.mp4'

# 저장된 encoding 파일
encoding_file = '/content/encodings.pickle'

# 이름 모를 경우 Unknwon 지정
unknown_name = 'Unknown'

# model 지정, cnn or hog
model_method = 'cnn'

# Detection 된 output 동영상
output_name = 'output_video2.mp4'

In [ ]:
def detectAndDisplay(image):
    start_time = time.time()

    # 이미지 RGB 형식으로 변경
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 얼굴 detection, boxing
    boxes = face_recognition.face_locations(rgb, model = model_method)

    # 얼굴영역 특성값 계산
    encodings = face_recognition.face_encodings(rgb, boxes)

    names = []

    for encoding in encodings:
        # boxing 된 특성이 pickle 파일의 encodings(특성값)과 매칭되는 것 찾기
        matches = face_recognition.compare_faces(data['encodings'], encoding)
        print(matches)
        name = unknown_name     # 'Unknown' 으로 초기화

        # check to see if we have found a match
        if True in matches:
            # 매칭된 pickle 파일의 True 인덱스 번호를 찾아온다
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            for i in matchedIdxs:
                name = data['names'][i]
                counts[name] = counts.get(name, 0) + 1

            name = max(counts, key = counts.get)

        # names 배열에 이름 추가
        names.append(name)

    # detection 된 얼굴의 boxing 위치 좌표와 이름 매칭
    for ((top, right, bottom, left), name) in zip(boxes, names):
        color = (0, 255, 0)
        line = 2
        if(name == unknown_name):
            color = (0, 0, 255)
            line = 1
            name = ''

        cv2.rectangle(image, (left, top), (right, bottom), color, line)
        y = top - 15 if top -15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, line)

    end_time = time.time()
    process_time = end_time - start_time
    print('= = = A frame took {:.3f} seconds'.format(process_time))

    # video를 disk에 output 하기 위해 writer 초기화
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        writer = cv2.VideoWriter(output_name, fourcc, 30, (image.shape[1], image.shape[0]), True)

    # disk 에 frame을 write 한다.
    if writer is not None:
        writer.write(image)

In [ ]:
# 저장된 encoding 파일 불러오기
data = pickle.loads(open(encoding_file, 'rb').read())
cap = cv2.VideoCapture(file_name)   # 원본 동영상에서 video stream 읽어오기
writer = None

if not cap.isOpened:
    print('- -(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()

        print('- -(!) No captured frame - - Break!')
        break
    detectAndDisplay(frame)